In [38]:
import statsmodels

import numpy as np
import scipy as sc
from matplotlib import pyplot as plt

from statsmodels.stats.proportion import proportion_confint, proportions_ztest, \
confint_proportions_2indep, test_proportions_2indep

In [41]:
import numpy as np
import pandas as pd
import scipy as sc
%pylab inline

from statsmodels.stats.weightstats import DescrStatsW, CompareMeans

Populating the interactive namespace from numpy and matplotlib


In [10]:
print("Wald's interval:", confint_proportions_2indep(189, 11034, 104, 11037, method = 'wald', compare='diff'))

Wald's interval: (0.004687750675049438, 0.010724297276960126)


In [11]:
test_proportions_2indep(189, 11034, 104, 11037, alternative = 'two-sided', method = 'score')

<class 'statsmodels.stats.base.HolderTuple'>
statistic = 5.00127490046852
pvalue = 5.69524362691927e-07
compare = diff
method = score
variance = 2.374101369219409e-06
alternative = two-sided
prop1_null = 0.013275338679715464
prop2_null = 0.013275338679715464
tuple = (5.00127490046852, 5.69524362691927e-07)
diff = 0.0077060239760047815
ratio = 1.8178017944535074
odds_ratio = 1.8320539419087138
value = 0

In [13]:
x = (189/11034)/ (1-189/11034) / (  (104/11037) / (1-104/11037))
x

1.8320539419087138

In [29]:
def get_bootstrap_samples(x, n_resamples):
    indices = np.random.randint(0, len(x), (n_resamples, len(x)))
    resamples = x[indices]
    return resamples

In [30]:
def percentile_interval(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [31]:
sample1 = np.array([1] * 104 + [0] * 10933)
sample2 = np.array([1] * 189 + [0] * 10845)

In [32]:
np.random.seed(0)

sample1_scores = list(map(np.median, get_bootstrap_samples(sample1, 1000)))
sample2_scores = list(map(np.median, get_bootstrap_samples(sample2, 1000)))

print("95% confidence interval for sample1", percentile_interval(sample1_scores, x))
print("95% confidence interval for sample2", percentile_interval(sample2_scores, x))

95% confidence interval for sample1 [0. 0.]
95% confidence interval for sample2 [0. 0.]


In [39]:
z = (9.57 - 9.5) / (0.4 / np.sqrt(160))
p = 2*(1-statsmodels.stats.norm.cdf(abs(z)))

AttributeError: module 'statsmodels.stats' has no attribute 'norm'

In [44]:
data = pd.read_csv('data/diamond_prices.csv', sep = ',', header = 0)

In [45]:
data.head()

,price,predicted_price_lm,predicted_price_gbdt
0,564,747.149466,799.049243
1,5914,6644.569397,6635.491541
2,2562,2096.573114,2138.584133
3,537,727.031366,694.107710
4,5964,7180.988674,7425.353560


In [53]:
lm_val = 0
gbdt_val = 0

for index, row in data.iterrows():
    lm_val += abs(row['price'] - row['predicted_price_lm'])
    gbdt_val += abs(row['price'] - row['predicted_price_gbdt'])

lm_result = lm_val / data.price.count()
gbdt_result = gbdt_val / data.price.count()

In [56]:
abs(lm_result - gbdt_result)

6.875650288841371

In [58]:
abs(data['price'] - data['predicted_price_lm']).mean() - abs(data['price'] - data['predicted_price_gbdt']).mean()

6.875650288835004

In [59]:
cm = CompareMeans(DescrStatsW(data.predicted_price_lm), DescrStatsW(data.predicted_price_gbdt))
print("95%% confidence interval: [%f, %f]" % cm.tconfint_diff(usevar='unequal'))

95% confidence interval: [-202.866133, -27.763321]
